In [ ]:
import pandas as pd
import os
import logging
import time
from sqlalchemy import create_engine

# Make sure 'logs/' folder exists
os.makedirs("logs", exist_ok=True)

# Setup logging
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.INFO,  # make sure it's INFO or DEBUG
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

engine = create_engine('sqlite:///inventory.db')

def ingest_db(df, table_name, engine):
    try:
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        logging.info(f"Uploaded: {table_name} | Shape: {df.shape}")
    except Exception as e:
        logging.error(f"Failed to ingest {table_name}: {e}")

def ingest_large_csv(file_path, table_name, engine, chunk_size=100_000):
    try:
        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
            mode = 'replace' if i == 0 else 'append'
            chunk.to_sql(table_name, con=engine, if_exists=mode, index=False)
            logging.info(f"{table_name} - Chunk {i+1} inserted: {chunk.shape}")
    except Exception as e:
        logging.error(f"Failed to ingest large file {table_name}: {e}")

def load_raw_data():
    start = time.time()
    data_folder = 'data'

    for file in os.listdir(data_folder):
        if file.endswith('.csv'):
            file_path = os.path.join(data_folder, file)
            table_name = file[:-4]

            if file == 'vendor_invoice.csv':
                ingest_large_csv(file_path, table_name, engine)
            else:
                try:
                    df = pd.read_csv(file_path)
                    ingest_db(df, table_name, engine)
                except Exception as e:
                    logging.error(f"Failed to read {file}: {e}")

    end = time.time()
    total_time = (end - start) / 60
    logging.info("-------------Ingestion Complete-------------")
    logging.info(f"Total Time Taken: {total_time:.2f} minutes")

if __name__ == '__main__':
    load_raw_data()


In [2]:
import pandas as pd
import os
import time
from sqlalchemy import create_engine

engine = create_engine('sqlite:///inventory.db')

def ingest_db(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Uploaded: {table_name} | Shape: {df.shape}")

def ingest_large_csv(file_path, table_name, engine, chunk_size=100_000):
    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
        mode = 'replace' if i == 0 else 'append'
        chunk.to_sql(table_name, con=engine, if_exists=mode, index=False)
        print(f"{table_name} - Chunk {i+1} inserted: {chunk.shape}")

def load_raw_data():
    start = time.time()
    data_folder = 'data'

    heavy = {'vendor_invoice.csv', 'sales.csv', 'purchases.csv'}
    for file in os.listdir(data_folder):
        if file.endswith('.csv'):
            path = os.path.join(data_folder, file)
            table = file[:-4]
            if file in heavy:
                print(f"Chunk loading heavy file: {file}")
                ingest_large_csv(path, table, engine)
            else:
                df = pd.read_csv(path)
                ingest_db(df, table, engine)

    dt = (time.time() - start) / 60
    print(f"\nIngestion done in {dt:.2f} mins")

# Just call load_raw_data() directly
load_raw_data()


Uploaded: begin_inventory | Shape: (206529, 9)
Uploaded: end_inventory | Shape: (224489, 9)
Chunk loading heavy file: purchases.csv
purchases - Chunk 1 inserted: (100000, 16)
purchases - Chunk 2 inserted: (100000, 16)
purchases - Chunk 3 inserted: (100000, 16)
purchases - Chunk 4 inserted: (100000, 16)
purchases - Chunk 5 inserted: (100000, 16)
purchases - Chunk 6 inserted: (100000, 16)
purchases - Chunk 7 inserted: (100000, 16)
purchases - Chunk 8 inserted: (100000, 16)
purchases - Chunk 9 inserted: (100000, 16)
purchases - Chunk 10 inserted: (100000, 16)
purchases - Chunk 11 inserted: (100000, 16)
purchases - Chunk 12 inserted: (100000, 16)
purchases - Chunk 13 inserted: (100000, 16)
purchases - Chunk 14 inserted: (100000, 16)
purchases - Chunk 15 inserted: (100000, 16)
purchases - Chunk 16 inserted: (100000, 16)
purchases - Chunk 17 inserted: (100000, 16)
purchases - Chunk 18 inserted: (100000, 16)
purchases - Chunk 19 inserted: (100000, 16)
purchases - Chunk 20 inserted: (100000, 1

In [3]:
from sqlalchemy import inspect

inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables found in inventory.db:")
print(tables)


Tables found in inventory.db:
['begin_inventory', 'end_inventory', 'purchase_prices', 'purchases', 'sales', 'vendor_invoice']


In [4]:
df = pd.read_sql("SELECT * FROM sales LIMIT 5", con=engine)
print(df)


           InventoryId  Store  Brand                 Description        Size  \
0  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
1  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
2  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
3  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
4  1_HARDERSFIELD_1005      1   1005     Maker's Mark Combo Pack  375mL 2 Pk   

   SalesQuantity  SalesDollars  SalesPrice   SalesDate  Volume  \
0              1         16.49       16.49  2024-01-01   750.0   
1              2         32.98       16.49  2024-01-02   750.0   
2              1         16.49       16.49  2024-01-03   750.0   
3              1         14.49       14.49  2024-01-08   750.0   
4              2         69.98       34.99  2024-01-09   375.0   

   Classification  ExciseTax  VendorNo                   VendorName  
0               1       0.79     12546  JIM BEAM BRA

In [5]:
for table in inspector.get_table_names():
    count = pd.read_sql(f"SELECT COUNT(*) AS rows FROM {table}", con=engine)
    print(f"{table}: {count['rows'][0]} rows")


begin_inventory: 206529 rows
end_inventory: 224489 rows
purchase_prices: 12261 rows
purchases: 2372474 rows
sales: 12825363 rows
vendor_invoice: 5543 rows
